In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re
import os
os.environ['CUDA_SET_VISIBLE_DEVICES']='1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH']='true'
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import L1L2

In [2]:
X_train = np.load('data/X_train.npy')
X_val = np.load('data/X_val.npy')
y_train = np.load('data/y_train.npy')
y_val = np.load('data/y_val.npy')

In [3]:
l1_grid = [0.01, 0.05, 0.1, 0.15, 0.2]
l2_grid = [0.01, 0.05, 0.1, 0.15, 0.2]
grid_params = []
for l1 in l1_grid:
    for l2 in l2_grid:
            grid_params.append((l1, l2))

In [8]:
def lr_scheduler(epoch, lr):
    if epoch <= 20:
        return 1e-1
    elif epoch > 20 and epoch <= 25:
        return 1e-2
    elif epoch > 25:
        return 1e-3

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [31]:
model_vl_loss = {}
model_history = {}
for i, params in enumerate(grid_params):
    print(f'Training model {i}')
    model = Sequential([Dense(1, input_shape=[X_train.shape[1]], activation='linear',
                        kernel_regularizer=L1L2(params[0], params[1]))])
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(x=X_train, y=y_train, batch_size=50000, epochs=50, validation_data=(X_val, y_val),
                        validation_batch_size=50000, callbacks=[lr_callback], verbose=0)
    key_str = str(params[0]) + '_' + str(params[1])
    model_vl_loss[key_str] = history.history['val_loss']
    model_history[key_str] = history.history

Training model 0
Training model 1
Training model 2
Training model 3
Training model 4
Training model 5
Training model 6
Training model 7
Training model 8
Training model 9
Training model 10
Training model 11
Training model 12
Training model 13
Training model 14
Training model 15
Training model 16
Training model 17
Training model 18
Training model 19
Training model 20
Training model 21
Training model 22
Training model 23
Training model 24


In [32]:
model_vl_loss

{'0.01_0.01': [0.364575058221817,
  0.3332725763320923,
  0.32876262068748474,
  0.32927313446998596,
  0.3289576768875122,
  0.32955124974250793,
  0.3335130214691162,
  0.329637736082077,
  0.32831838726997375,
  0.33583876490592957,
  0.33014604449272156,
  0.32832807302474976,
  0.3297012746334076,
  0.33001428842544556,
  0.3315638601779938,
  0.3289293944835663,
  0.3317902684211731,
  0.3325609862804413,
  0.3300933837890625,
  0.33014369010925293,
  0.330509215593338,
  0.32781171798706055,
  0.32754912972450256,
  0.32766884565353394,
  0.3276463747024536,
  0.3277924060821533,
  0.3275317847728729,
  0.3274790346622467,
  0.3274441063404083,
  0.32751408219337463,
  0.32749202847480774,
  0.3274538516998291,
  0.327480286359787,
  0.3274881839752197,
  0.3274647295475006,
  0.3274514675140381,
  0.3274385631084442,
  0.3274332880973816,
  0.3274701237678528,
  0.3274572789669037,
  0.3274565041065216,
  0.32745033502578735,
  0.3274523913860321,
  0.3274548649787903,
  0.3274

In [33]:
tmp_vl_loss = {}
for key in model_vl_loss.keys():
    tmp_vl_loss[key] = model_vl_loss[key][-1]

In [34]:
tmp_vl_loss

{'0.01_0.01': 0.32746177911758423,
 '0.01_0.05': 0.3277563452720642,
 '0.01_0.1': 0.3281152844429016,
 '0.01_0.15': 0.3284189999103546,
 '0.01_0.2': 0.32871052622795105,
 '0.05_0.01': 0.3307485282421112,
 '0.05_0.05': 0.33094698190689087,
 '0.05_0.1': 0.3311136066913605,
 '0.05_0.15': 0.33127716183662415,
 '0.05_0.2': 0.3314690589904785,
 '0.1_0.01': 0.333569198846817,
 '0.1_0.05': 0.3335891366004944,
 '0.1_0.1': 0.33370763063430786,
 '0.1_0.15': 0.33376985788345337,
 '0.1_0.2': 0.333786278963089,
 '0.15_0.01': 0.3350352942943573,
 '0.15_0.05': 0.33496060967445374,
 '0.15_0.1': 0.33498290181159973,
 '0.15_0.15': 0.3349771201610565,
 '0.15_0.2': 0.3351120948791504,
 '0.2_0.01': 0.3351916968822479,
 '0.2_0.05': 0.3351134657859802,
 '0.2_0.1': 0.33530375361442566,
 '0.2_0.15': 0.33534151315689087,
 '0.2_0.2': 0.33512383699417114}

In [37]:
min(tmp_vl_loss, key=tmp_vl_loss.get)

'0.01_0.01'